# Hoeveel te bestellen.

In [1]:
import pandas as pd
import math

# Helper functions.
def get_item_info():
    item_info=pd.DataFrame({'Item':['Sensor', 'Infuusset', 'Reservoir'],
                    'Days per unit':[7, 3, 3],
                    'Units in package':[5,10,10],
                    'Official name':['Guardian Sensor 4', 'Mio Advance 6mm 60cm', 'Medtr reservoir 3ml']
                   })
    return item_info

def get_specifics(item_info, item):
    specs = item_info.loc[item_info['Item'] == item].iloc[0]
    days_per_unit, units_in_package, official_name = specs['Days per unit'], specs['Units in package'], specs['Official name']
    
    assert days_per_unit > 0 and units_in_package > 0, "Days per unit and units in package should be positive."
    
    return days_per_unit, units_in_package, official_name

def until_what_date_covered(stock:int,
                            days_per_unit:float,
                            units_in_package:int,
                            order:int=0,
                            margin:float=0.2,
                            verbose:bool = True):
    
    days_covered = (stock*days_per_unit + order*units_in_package*days_per_unit)*(1-margin)
    final_date = pd.to_datetime('today') + pd.DateOffset(days=days_covered)
    final_date = final_date.strftime("%d-%m-%Y")
    
    if verbose:
        addition = ' including order' if order>0 else ''
        print(f'{int(days_covered)} days are covered by stock{addition}.')
    return final_date

In [2]:
def assess_status(item: str = ["Sensor", "Infuusset", "Reservoir"],
                 till_date: str = None,
                nr_of_days: float = 0,
                 stock: int = 0,
                 order: int = 0,
                 margin: float = 0.2,
                 verbose: bool = True,
                 item_info: pd.DataFrame = None):
    """
    Calculate how many packages I need to order of a specific item, or whether the current order is sufficient,
    or until what date the current stock or order provides.

    Parameters
    ----------
    item
        The diabetes item to check. One of ['Sensor', 'Infuusset', 'Reservoir'] for default item info.
    till_date
        Until what date do I want to make sure to have enough supply? Give in the form 'YYYY-MM-DD'.
    nr_of_days
        For how many days do I want to make sure to have enough supply?
    stock
        How many units do I've got in stock of this specific item?
    order
        If I want to check the current order amount: Fill in number of packages in the order.
    margin
        To prevent living on the edge: what's the margin to cover for item failings? As fraction.
    verbose
        Print output yes or no.
    item_info
        If I want to give a different dataframe than given in the function, give here.
        Must follow certain structure.
        Check the `item_info` dataframe in the function code for this structure.
    """
    
    # Check input.
    assert margin >= 0, "Margin should be non-negative."
    assert stock >= 0, "Stock should be non-negative."
    assert nr_of_days >= 0, "Number of days should be non-negative."
    assert pd.to_datetime(till_date, errors='coerce') is not pd.NaT, "Invalid format for till_date. Should adhere to 'YYYY-MM-DD'."
    
    # Product info.
    if item_info is None:
        item_info = get_item_info()
    assert item_info is not None and not item_info.empty, "Item info DataFrame must be provided and non-empty."

    # Get the item's specifics.
    days_per_unit, units_in_package, official_name = get_specifics(item_info, item)

    # Determine assignment.
    if till_date is None and nr_of_days == 0:
        option = 1
    elif till_date is not None or nr_of_days > 0:
        if order == 0:
            option = 2
        elif order > 0:
            option = 3
    else:
        # TODO break
        print("errorrr does not exist.")

    # Print item info + assignment.
    if verbose:
        assignments = {1:"Checking until when diabetes needs are covered...",
                       2:"Calculating how many packages are needed...",
                       3:f"Checking if an order with {order} package(s) is enough..."}
        print(f'-- Item: {item} --\n\nOfficial name: {official_name}.\n{stock} units in stock.')
        lines='\n----------------------\n'
        print(lines)
        print(assignments[option])
        print(lines)

    if option == 1:
        # Calculate how many days are covered.
        date_stock = until_what_date_covered(stock=stock, days_per_unit=days_per_unit,
                                             units_in_package=units_in_package,
                                             order=0,
                                             margin=margin, verbose=verbose)
        verdict = f'Current stock covers until {date_stock}.'
            
        if order>0:
            date_order = until_what_date_covered(stock=stock, days_per_unit=days_per_unit,
                                             units_in_package=units_in_package, order=order,
                                                margin=margin, verbose=verbose)
            verdict = verdict + f' Including order, the items cover until {date_order}.'

    elif option in [2, 3]:
        # Calculate how many days need to be covered.
        if till_date is not None:
            till_date, today = pd.to_datetime(till_date), pd.to_datetime('today')
            days_needed = (till_date-today).days*(1+margin)
            if verbose:
                print(f'Starting date: {today.date()}\nCovering until date: {till_date.date()}\n{round(days_needed,1)} days to be covered.\n{lines}')
        if nr_of_days>0:
            days_needed = nr_of_days
    
        # Calculate.
        days_covered=stock*days_per_unit + order*units_in_package*days_per_unit
        days_lacking=days_needed-days_covered
        more_to_order=math.ceil(days_lacking/days_per_unit/units_in_package)
        
        if verbose:
            print(f'{int(days_covered)} days are covered.')
            if days_lacking<=0:
                print(f'Stock covers enough days with {abs(int(days_lacking))} extra days.\n')
            else:
                print(f'{int(days_lacking)} days are not covered in this order.\n' if order>0 else f'{int(days_lacking)} more days need to be covered.\n')
        
        # State how much more is needed.
        if order>0:
            verdict='Order is sufficient' if days_lacking<=0 else f'Not enough: {more_to_order} more package(s) needed.'
        else:
            verdict=f"{'Stock is sufficient' if days_lacking<=0 else f'Order must contain at least {more_to_order} package(s).'}"
    else:
        print("errorrorror smth's wrong")
    
    return verdict

## Tests

In [7]:
assess_status(item = "Sensor",
                 till_date = None,
                nr_of_days = 0,
                 stock = 10,
                 order = 0,
                 margin = 0.2,
                 verbose = True,
                 item_info = None)

-- Item: Sensor --

Official name: Guardian Sensor 4.
10 units in stock.

----------------------

Checking until when diabetes needs are covered...

----------------------

56 days are covered by stock.


'Current stock covers until 14-06-2024.'

In [110]:
assess_status(item = "Infuusset",
                 till_date = None,
                nr_of_days = 0,
                 stock = 10,
                 order = 0,
                 margin = 0.2,
                 verbose = True,
                 item_info = None)

-- Item: Infuusset --

Official name: Mio Advance 6mm 60cm.
10 units in stock.

----------------------

Checking until when diabetes needs are covered...

----------------------

24 days are covered by stock.


'Current stock covers until 07-05-2024.'

In [111]:
assess_status(item = "Sensor",
                 till_date = None,
                nr_of_days = 14,
                 stock = 10,
                 order = 0,
                 margin = 0.2,
                 verbose = True,
                 item_info = None)

-- Item: Sensor --

Official name: Guardian Sensor 4.
10 units in stock.

----------------------

Calculating how many packages are needed...

----------------------

70 days are covered.
Stock covers enough days with 56 extra days.



'Stock is sufficient'

In [112]:
assess_status(item = "Sensor",
                 till_date = None,
                nr_of_days = 0,
                 stock = 10,
                 order = 0,
                 margin = 0.2,
                 verbose = True,
                 item_info = None)

-- Item: Sensor --

Official name: Guardian Sensor 4.
10 units in stock.

----------------------

Checking until when diabetes needs are covered...

----------------------

56 days are covered by stock.


'Current stock covers until 08-06-2024.'

In [113]:
assess_status(item = "Sensor",
                 till_date = None,
                nr_of_days = 0,
                 stock = 10,
                 order = 0,
                 margin = 0.2,
                 verbose = False,
                 item_info = None)

'Current stock covers until 08-06-2024.'

In [114]:
assess_status(item = "Sensor",
                 till_date = None,
                nr_of_days = 0,
                 stock = 10,
                 order = 0,
                 margin = 0.6,
                 verbose = True,
                 item_info = None)

-- Item: Sensor --

Official name: Guardian Sensor 4.
10 units in stock.

----------------------

Checking until when diabetes needs are covered...

----------------------

28 days are covered by stock.


'Current stock covers until 11-05-2024.'

In [117]:
assess_status(item = "Infuusset",
                 till_date = '2024-06-28',
                nr_of_days = 0,
                 stock = 10,
                 order = 0,
                 margin = 0.2,
                 verbose = True,
                 item_info = None)

-- Item: Infuusset --

Official name: Mio Advance 6mm 60cm.
10 units in stock.

----------------------

Calculating how many packages are needed...

----------------------

Starting date: 2024-04-13
Covering until date: 2024-06-28
90.0 days to be covered.

----------------------

30 days are covered.
60 more days need to be covered.



'Order must contain at least 2 package(s).'

In [118]:
assess_status(item = "Infuusset", stock = 5)

-- Item: Infuusset --

Official name: Mio Advance 6mm 60cm.
5 units in stock.

----------------------

Checking until when diabetes needs are covered...

----------------------

12 days are covered by stock.


'Current stock covers until 25-04-2024.'

In [120]:
assess_status(item = "Sensor", stock = 10)

-- Item: Sensor --

Official name: Guardian Sensor 4.
10 units in stock.

----------------------

Checking until when diabetes needs are covered...

----------------------

56 days are covered by stock.


'Current stock covers until 08-06-2024.'

In [6]:
assess_status(item = "Infuusset", stock = 12, margin=0)

-- Item: Infuusset --

Official name: Mio Advance 6mm 60cm.
12 units in stock.

----------------------

Checking until when diabetes needs are covered...

----------------------

36 days are covered by stock.


'Current stock covers until 07-08-2024.'

In [5]:
assess_status(item = "Reservoir",
                 till_date = '2024-09-25',
                 stock = 12,
                 margin = 0,
                 verbose = True)

-- Item: Reservoir --

Official name: Medtr reservoir 3ml.
12 units in stock.

----------------------

Calculating how many packages are needed...

----------------------

Starting date: 2024-07-02
Covering until date: 2024-09-25
84 days to be covered.

----------------------

36 days are covered.
48 more days need to be covered.



'Order must contain at least 2 package(s).'

## Deprecated

In [ ]:
def how_many_needed(item: str = ["Sensor", "Infuusset", "Reservoir"],
                 till_date: str = None,
                    nr_of_days: float = None,
                 stock: int = 0,
                 order: int = 0,
                 margin: float = 0.2,
                 verbose: bool = True,
                 item_info: pd.DataFrame = None
                        ):
    """
    Calculate how many packages I need to order of a specific item, or whether the current order is sufficient.

    Parameters
    ----------
    item
        The diabetes item to check. One of ['Sensor', 'Infuusset', 'Reservoir'].
    till_date
        Until what date do I want to make sure to have enough supply? Give in the form 'YYYY-MM-DD'.
    nr_of_days
        For how many days do I want to make sure to have enough supply?
    stock
        How many units do I've got in stock of this specific item?
    order
        If I want to check the current order amount: Fill in number of packages in the order.
    margin
        To prevent living on the edge: what's the margin to cover for item failings? As fraction.
    verbose
        Print output yes or no.
    item_info
        If I want to give a different dataframe than given in the function, give here.
        Must follow certain structure.
        Check the `item_info` dataframe in the function code for this structure.
    """

    # Check input.
    assert margin >= 0, "Margin should be non-negative."
    assert stock >= 0, "Stock should be non-negative."
    assert pd.to_datetime(till_date, errors='coerce') is not pd.NaT, "Invalid format for till_date. Should adhere to 'YYYY-MM-DD'."
    assert nr_of_days is not None or till_date is not None, "Give either the number of days you want to cover or until which date you want to be covered."

    # Background info.
    if item_info is None:
        item_info = get_item_info()
    assert item_info is not None and not item_info.empty, "Item info DataFrame must be provided and non-empty."
    
    # Get the item's specifics.
    days_per_unit, units_in_package, official_name = get_specifics(item_info, item)
    
    if verbose:
        print(f'-- Item: {item} --\n\nOfficial name: {official_name}.\n{stock} units in stock.')
        if order>0:
            print(f'\nChecking if an order with {order} package(s) is enough...')
        else:
            print('\nCalculating how many packages are needed...')
        lines='----------------------'
        print(lines)
    
    # Calculate how many days need to be covered.
    if till_date is not None:
        till_date, today = pd.to_datetime(till_date), pd.to_datetime('today')
        days_needed = (till_date-today).days*(1+margin)
        if verbose:
            print(f'Starting date: {today.date()}\nCovering until date: {till_date.date()}\n{round(days_needed,1)} days to be covered.\n{lines}')
    if nr_of_days is not None:
        days_needed = nr_of_days

    # Calculate.
    days_covered=stock*days_per_unit + (order*units_in_package*days_per_unit if order>0 else 0)
    days_lacking=days_needed-days_covered
    more_to_order=math.ceil(days_lacking/days_per_unit/units_in_package)
    
    if verbose:
        print(f'{int(days_covered)} days are covered.')
        if days_lacking<=0:
            print(f'Stock covers enough days with {abs(int(days_lacking))} extra days.\n')
        else:
            print(f'{int(days_lacking)} days are not covered in this order.\n' if order>0 else f'{int(days_lacking)} more days need to be covered.\n')
    
    # State how much more is needed.
    if order>0:
        verdict='Order is sufficient' if days_lacking<=0 else f'Not enough: {more_to_order} more package(s) needed.'
    else:
        verdict=f"{'Stock is sufficient' if days_lacking<=0 else f'Order must contain at least {more_to_order} package(s).'}"

    return verdict

In [ ]:
def until_when_covered(item: str = ["Sensor", "Infuusset", "Reservoir"],
                 stock: int = 0,
                 order: int = 0,
                 margin: float = 0.2,
                 verbose: bool = True,
                 item_info: pd.DataFrame = None):
    """

    """
    
    # Check input.
    assert margin >= 0, "Margin should be non-negative."
    assert stock >= 0, "Stock should be non-negative."

    # Background info.
    if item_info is None:
        item_info = get_item_info()
    assert item_info is not None and not item_info.empty, "Item info DataFrame must be provided and non-empty."
    
    # Get the item's specifics.
    days_per_unit, units_in_package, official_name = get_specifics(item_info, item)
    
    if verbose:
        print(f'-- Item: {item} --\n\nOfficial name: {official_name}.\n{stock} units in stock.')
        lines='\n----------------------\n'
        print(lines)
        if order>0:
            print(f'Checking until when stock+order covers diabetes needs...')
        else:
            print('Checking until when stock covers diabetes needs...')
        print(lines)
    
    # Calculate how many days are covered.
    date_stock = until_what_date_covered(stock=stock, days_per_unit=days_per_unit,
                                         units_in_package=units_in_package,
                                         order=0,
                                         margin=margin, verbose=verbose)
    verdict = f'Current stock covers until {date_stock}.'
        
    if order>0:
        date_order = until_what_date_covered(stock=stock, days_per_unit=days_per_unit,
                                         units_in_package=units_in_package, order=order,
                                            margin=margin, verbose=verbose)
        verdict = verdict + f' Including order, the items cover until {date_order}.'

    return verdict